In [1]:
import pandas as pd
import dask.dataframe as dd
import dask.bag as db
from gensim.models import Word2Vec
docs = dd.read_parquet("output.pq/")

In [4]:
texts = docs['News_Tokens'].compute()
bag = db.from_sequence(texts)
sentences = bag.flatten().compute()

In [ ]:
print(sentences[:10])

['май состояться общенародный акция бессмертный полк который принять участие студент сотрудник', 'два год ограничение пандемия долгожданный шествие проходить традиционный формат', 'время построение колонна корпус задавать настроение концертный программа', 'звучать песня военный год исполнить студент участник', 'зритель узнавать горячо любимый песня подпевать дарить свой аплодисменты', 'вокальный номер перемежаться танцевальный', 'ансамбль современный хореография представить выступление который отразить многонациональность культурный многообразие', 'пересечение улица колонна начало свой движение мелодичный звук оркестр', 'участник акция бережно нести портрет свой родственник который приближать победа поле бой тыл портрет пасть герой великий отечественный война прадед дед отвоевать мир следующий поколение', 'человек делиться свой радость петь песня смуглянка синий платочек воодушевлённо кричать ура']


In [7]:
model = Word2Vec(sentences=sentences, vector_size=100, window=5,min_count=1, sg=0)
model.save('word2vec_sent_5_sgu.model')

# Построение тезауруса

In [7]:
import pandas as pd
import dask.dataframe as dd
import dask.bag as db
from gensim.models import Word2Vec
docs = dd.read_parquet("output.pq/")

In [14]:
model = Word2Vec.load("word2vec_sent_5_sgu.model")
texts = docs['News_Tokens'].compute()
bag = db.from_sequence(texts)
sentences = bag.flatten()

In [17]:
words = sentences.map(lambda x: x.split()).flatten().distinct().compute()


In [19]:
print(len(words))

45204


In [ ]:
results = []
threshold = 0.7
for word in words:
    try:
        # Получаем 10 ближайших слов
        similar_words = model.wv.most_similar(word, topn=10)
        filtered_words = [(w, sim) for w, sim in similar_words if sim >= threshold]
        results.append([word, similar_words])
    except KeyError:
        # Если слово не в модели, пропускаем его
        print(f"Слово '{word}' не найдено в модели.")

df_results = pd.DataFrame(results, columns=["Word", "Most_Similar_Word"])

df_results.to_csv("similar_words10.csv", index=False)